# Sample Pipeline

This notebook shows an end to end radio interferometry pipeline from the simulation of the sky to the final image. In addition, it produces a catalogue of the sources in the image pixel domain. This is just an example notebook and thus the functions are not well structured and capsulated yet.

The pipeline consists of three modules:

- Simulation
    - Sky module: OSKAR
    - Telescope module incl. calibration: OSKAR
- Processing
    - Calibration after observation: RASCIL
    - Deconvolution: RASCIL
- Analysis & comparison
    - Quantitative and qualitative analysis of algorithms

In [1]:
import sys
import oskar
import matplotlib
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import numpy as np

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy import wcs

## Simulation

The sky and telescope simulation is currently provided completely by OSKAR.

### Sky Module

The sky module of OSKAR contains radiation sources, which are defined as array and can be passed to `oskar.Sky.from_array`.

In [3]:
import pandas as pd

df_racs = pd.read_csv('AS110_Derived_Catalogue_racs_dr1_gaussians_galacticregion_v2021_08_v01_4550.csv')
df_racs.head()

,id,catalogue_id,gaussian_id,source_id,tile_id,sbid,obs_start_time,n_gaus,ra,dec,...,e_dc_maj,dc_min,e_dc_min,dc_pa,e_dc_pa,s_code,separation_tile_centre,noise,gal_lon,gal_lat
0,1,4550,RACS_0526+25A_1201,RACS_0526+25A_1011,RACS_0526+25A,13722,58971.25398,1,84.893217,24.416944,...,6.63,12.29,4.95,101.15,53.81,S,3.1529,0.471,183.136184,-3.524673
1,2,4550,RACS_0526+25A_1206,RACS_0526+25A_1016,RACS_0526+25A,13722,58971.25398,1,84.869190,23.933755,...,6.26,0.00,3.92,0.00,35.34,S,3.2804,0.408,183.535265,-3.799150
2,3,4550,RACS_0526+25A_1212,RACS_0526+25A_1020,RACS_0526+25A,13722,58971.25398,1,84.900889,25.511269,...,1.84,2.68,1.80,80.56,1.46,S,3.0898,0.449,182.210449,-2.938719
3,4,4550,RACS_0526+25A_1213,RACS_0526+25A_1021,RACS_0526+25A,13722,58971.25398,1,84.878595,24.759065,...,1.80,0.00,1.71,0.00,129.97,S,3.0767,1.340,182.838456,-3.354672
4,5,4550,RACS_0526+25A_1214,RACS_0526+25A_1022,RACS_0526+25A,13722,58971.25398,1,84.897391,25.460482,...,5.57,8.76,4.20,103.73,50.90,S,3.0814,0.404,182.251884,-2.968334


In [4]:
df_racs.columns

Index(['id', 'catalogue_id', 'gaussian_id', 'source_id', 'tile_id', 'sbid',
       'obs_start_time', 'n_gaus', 'ra', 'dec', 'e_ra', 'e_dec',
       'total_flux_gaussian', 'e_total_flux_gaussian_pybdsf',
       'e_total_flux_gaussian', 'total_flux_source',
       'e_total_flux_source_pybdsf', 'e_total_flux_source', 'peak_flux',
       'e_peak_flux', 'maj', 'e_maj', 'min', 'e_min', 'pa', 'e_pa', 'dc_maj',
       'e_dc_maj', 'dc_min', 'e_dc_min', 'dc_pa', 'e_dc_pa', 's_code',
       'separation_tile_centre', 'noise', 'gal_lon', 'gal_lat'],
      dtype='object')

Instead of using completely artificial sources, an external catalog can also be used. Here we use the GLEAM survey, which can be downloaded from the [VizieR](https://cdsarc.unistra.fr/viz-bin/cat/VIII/100) service. 

This example is using the [GLEAM EGC catalog, version 2](https://vizier.cds.unistra.fr/viz-bin/VizieR-3). To download the full survey, you have to change **preferences/max** to unlimited. The download here was done using the **FITS (binary) Table** option which can be selected in the dropdown below.

For this example we only use right ascension (RAJ2000) and declination (DEJ2000) and the Stokes I peak flux intensity at 76 MHz (Fp076). There are some sources which have no flux values in the corresponding frequency band. These must be removed first.

In [5]:
from datetime import timedelta, datetime
import sys
sys.path.insert(1, '/home/lukas/i4ds/ska/KaraboPipeline')
from karabo.simulation.sky_model import *
from karabo.simulation.observation import *
from karabo.simulation.utils import *
from karabo.simulation.telescope import *
from karabo.simulation.observation import *
from karabo.simulation.interferometer import *
from karabo.simulation.imager import *

ImportError: libboost_python39.so.1.73.0: cannot open shared object file: No such file or directory

In [ ]:
gleam = SkyModel.get_fits_catalog('./GLEAM_EGC.fits')
df_gleam = gleam.to_pandas()
df_gleam.head()

In [ ]:
ref_freq = 76e6
start_frequency_hz = 76e6
df_gleam = df_gleam[~df_gleam['Fp076'].isna()]
ra, dec, fp = df_gleam['RAJ2000'], df_gleam['DEJ2000'], df_gleam['Fp076']
sky_array = np.column_stack((ra, dec, fp, np.zeros(ra.shape[0]), np.zeros(ra.shape[0]), 
                             np.zeros(ra.shape[0]), [ref_freq]*ra.shape[0])).astype('float32')
sky = SkyModel(sky_array)
sky[:,-1] = df_gleam['GLEAM']
sky.num_sources

Before further steps are taken, the phase center of the telescope must be defined.

In [ ]:
ra0 = 250
dec0 = -80
phase_center = [ra0,dec0]

In [ ]:
def plot_sky(sky, phase_center):
    ra0, dec0 = phase_center[0], phase_center[1]
    data = sky[:,0:3]
    ra = np.radians(data[:, 0] - ra0)
    dec = np.radians(data[:, 1])
    flux = data[:, 2]
    log_flux = np.log10(flux) # doesn't work as intended yet because of log of negative values
    x = np.cos(dec) * np.sin(ra)
    y = np.cos(np.radians(dec0)) * np.sin(dec) - \
                np.sin(np.radians(dec0)) * np.cos(dec) * np.cos(ra)
    sc = plt.scatter(x, y, s=.5, c=log_flux, cmap='plasma',
                vmin=np.min(log_flux), vmax=np.max(log_flux))
    plt.axis('equal')
    plt.xlabel('x direction cosine')
    plt.ylabel('y direction cosine')
    plt.colorbar(sc, label='Log10(Stokes I flux [Jy])')
    plt.show()
    
plot_sky(sky, phase_center)

Now that we have determined the phase center we can take a closer look at how the point sources are distributed around the center. For this we use a coordinate 2d image projection with the phase center as the origin using the the world coordinate system ([wcs](https://docs.astropy.org/en/stable/wcs/index.html)) of astropy. Specific information about the wcs parameters can be found [here](https://docs.astropy.org/en/stable/api/astropy.wcs.Wcsprm.html). For this illustration, we do not take into account the flux intensity, so that all sources are clearly visible.

In [ ]:
sky.explore_sky(phase_center=phase_center, figsize=(8,6))

Now, to have only a partition of the sky, we can use the `filter_by_radius`, which filters from the phase center with an inner and outer radius in degrees. Oskar also provides the function `filter_by_flux` to filter by Stokes-I flux. For more details we refer to the Oskar [documentation](https://fdulwich.github.io/oskarpy-doc/sky.html).

In [ ]:
# first filter sky by inner and outer radius in degrees from the phase center
sky.filter_by_radius(0, .55, phase_center[0], phase_center[1])
sky.num_sources

Now let's look at the filtered sources.

In [ ]:
sky.explore_sky(phase_center=phase_center, figsize=(10,8), s=80,
                xlim=(-.55,.55), ylim=(-.55,.55), with_labels=True)

### Telescope Module

Various observation parameters and meta information `params` must be passed to the telescope module `oskar.Interferometer` of OSKAR as `oskar.SettingsTree`.

In [ ]:
telescope = read_OSKAR_tm_file('../data/telescope.tm')

In [ ]:
observation = Observation(start_frequency_hz=start_frequency_hz, start_date_and_time=datetime(2000, 1, 1, 12, 0),
                          length=timedelta(hours=12), number_of_channels=16, frequency_increment_hz=20e6,
                          phase_centre_ra_deg=ra0, phase_centre_dec_deg=dec0, number_of_time_steps=24)

In [ ]:
interferometer = InterferometerSimulation(output_path='./visibilities', channel_bandwidth_hz=1e6)

In [ ]:
interferometer.run_simulation(telescope, sky, observation)

In [ ]:
# Basic settings. (Note that the sky model is set up later.)
#params = {
#    "simulator": {
#        "use_gpus": "False"
#    },
#    "observation" : {
#        #"num_channels": 64,
#        "num_channels": "16",
#        "start_frequency_hz": str(start_frequency_hz),
#        "frequency_inc_hz": "20e6",
#        "phase_centre_ra_deg": str(phase_center[0]),
#        "phase_centre_dec_deg": str(phase_center[1]),
#        "num_time_steps": "24",
#        "start_time_utc": "01-01-2000 12:00:00.000",
#        "length": "12:00:00.000"
#    },
#    "telescope": {
#        "input_directory": "../data/telescope.tm"
#    },
#    "interferometer": {
#        "ms_filename": "visibilities_gleam.ms",
#        "channel_bandwidth_hz": str(1e6),
#        "time_average_sec": "10"
#    }
#}
#settings = oskar.SettingsTree("oskar_sim_interferometer")
#settings.from_dict(params)

#precision = "single"
#if precision == "single":
#    settings["simulator/double_precision"] = "False"

# Set the sky model and run the simulation.
#sim = oskar.Interferometer(settings=settings)
#sim.set_sky_model(sky)
#sim.run()

### Observation Simulation

Now the sky module must be passed to the interferometer and the simulation of the observation must be started to generate the measurement set.

## Processing

After the observation is made with the telescope, a calibration of the measured data must be performed, followed by the reconstruction of the image.

### Calibration after Observation

toDo

### Imaging

Start an mmclean algorithm with the visibilites.ms as an input to deconvolve. 
To use dask cluster where you can see the progress, first create a dask cluster in the dask-extension on the left. 
Then copy the scheduler adress into the variable below. It might be correct already.

If you don't do this, remove the --dask_scheduler option from the options in the start_imager call.
Then RASCIL starts its own scheduler, you will however not be able to see the dashbaord, as the port is probably not forwarded by docker.

In [ ]:
#from rascil.apps import rascil_imager
#from rascil.processing_components.util.performance import (
#    performance_store_dict,
#    performance_environment,
#)
    
#def start_imager(rawargs):
#    parser = rascil_imager.cli_parser()
#    args = parser.parse_args(rawargs)
#    performance_environment(args.performance_file, mode="w")
#    performance_store_dict(args.performance_file, "cli_args", vars(args), mode="a")
#    image_name = rascil_imager.imager(args)

#start_imager(
#    [
#        '--ingest_msname','visibilities_gleam.ms',
#        '--ingest_dd', '0', 
#        '--ingest_vis_nchan', '16',
#        '--ingest_chan_per_blockvis', '1' ,
#        '--ingest_average_blockvis', 'True',
#        '--imaging_npixel', '2048', 
#        '--imaging_cellsize', '3.878509448876288e-05',
#        '--imaging_weighting', 'robust',
#        '--imaging_robustness', '-0.5',
#        '--clean_nmajor', '2' ,
#        '--clean_algorithm', 'mmclean',
#        '--clean_scales', '0', '6', '10', '30', '60',
#        '--clean_fractional_threshold', '0.3',
#        '--clean_threshold', '0.12e-3',
#        '--clean_nmoment' ,'5',
#        '--clean_psf_support', '640',
#        '--clean_restored_output', 'integrated'
#    ])

In [ ]:
imager = Imager(ingest_msname='visibilities_gleam.ms',
                ingest_dd=0,
                ingest_vis_nchan=16,
                ingest_chan_per_blockvis=1,
                ingest_average_blockvis=True,
                imaging_npixel=2048,
                imaging_cellsize=3.878509448876288e-05,
                imaging_weighting='robust',
                imaging_robustness=-.5,
                clean_nmajor=2,
                clean_algorithm='mmclean',
                clean_scales=[0,6,10,30,60],
                clean_fractional_threshold=.3,
                clean_threshold=.12e-3,
                clean_nmoment=5,
                clean_psf_support=640,
                clean_restored_output='integraed')

#imager.imaging_rascil()

## Analysis and Comparison

toDo

Now we have the image of the pipeline and the corresponding pixel coordinates as ground truth. Important are the number of set pixels and the cellsize, from which the coordinates are calculated.

In [ ]:
type(px)

In [ ]:
hdulist = fits.open('./visibilities_gleam_nmoment5_cip_deconvolved.fits')
w_fits = wcs.WCS(hdulist[0].header)

#### Construct Fits file ######
w = wcs.WCS(naxis=2)
w.wcs.crpix = w_fits.wcs.crpix[0:2] # coordinate reference pixel per axis --> on image
w.wcs.cdelt = w_fits.wcs.cdelt[0:2] # coordinate increments on sphere per axis
w.wcs.crval = [phase_center[0], phase_center[1]]
w.wcs.ctype = ["RA---AIR", "DEC--AIR"] # coordinate axis type
px, py = w.wcs_world2pix(sky[:,0], sky[:,1], 1) # coordinate conversion
fig, ax = plt.subplots()
ax.scatter(px, py)

for i, txt in enumerate(sky[:,-1]):
    ax.annotate(txt, (px[i], py[i]))
    pass
plt.show()

In [ ]:
import pandas as pd
px_coord = pd.DataFrame({'source':source_name, 'RAJ2000':ra_filtered, 'DEJ2000':dec_filtered, 'px':px, 'py':py, 'stokes_I_flux':df_gleam_filtered['Fp076']})
px_coord